In [78]:
import pandas as pd
from itertools import combinations
from collections import Counter

In [79]:
df = pd.read_csv('../datas/interim/matched_edits_all.csv')

In [80]:
def get_user_type(row):
    if any(c.isdigit() for c in row["user"].split(".") if row["user"].count(".") == 3):
        return "Anonymous (IP)"
    else:
        return "Registered"

df["user_type"] = df.apply(get_user_type, axis=1)

In [81]:
df["is_weaponised"] = df["weaponised"].apply(lambda x: 1 if str(x).lower().strip() == "weaponised" else 0)
weaponising_ratio_df = (
    df.groupby(["user", "article"])
      .agg(
          total_edits=("is_weaponised", "count"),
          weaponised_edits=("is_weaponised", "sum")
      )
      .reset_index()
)

weaponising_ratio_df["weaponising_ratio"] = (
    weaponising_ratio_df["weaponised_edits"] / weaponising_ratio_df["total_edits"]
)

weaponising_ratio_df

,user,article,total_edits,weaponised_edits,weaponising_ratio
0,*Kat*,2004_Ukrainian_presidential_election_analysis,3,0,0.0
1,.:Ajvol:.,Alexander_II_of_Russia_analysis,1,0,0.0
2,.:Ajvol:.,Eastern_Front_(World_War_II)_analysis,1,0,0.0
3,.:Ajvol:.,History_of_Ukraine_analysis,6,0,0.0
4,.anacondabot,Communist_Party_of_the_Soviet_Union_analysis,1,0,0.0
...,...,...,...,...,...
2873,Кокушев Сергей,Abortion_in_Ukraine_analysis,1,0,0.0
2874,Савелий В А,History_of_Ukraine_analysis,6,0,0.0
2875,УАВячеслав,Crimea_analysis,22,22,1.0
2876,Чръный человек,Epiphanius_I_of_Ukraine_analysis,2,0,0.0


In [86]:
weaponising_ratio_df["article_id"] = weaponising_ratio_df["article"].astype("category").cat.codes
weaponising_ratio_df["user_type"] = weaponising_ratio_df.apply(get_user_type, axis=1)
df_graph = (
    weaponising_ratio_df.groupby(["user", "article_id", "user_type", "weaponising_ratio"])
      .size()
      .reset_index(name="n_edits")
)
df_graph

,user,article_id,user_type,weaponising_ratio,n_edits
0,*Kat*,0,Registered,0.0,1
1,.:Ajvol:.,5,Registered,0.0,1
2,.:Ajvol:.,23,Registered,0.0,1
3,.:Ajvol:.,39,Registered,0.0,1
4,.anacondabot,16,Registered,0.0,1
...,...,...,...,...,...
2873,Кокушев Сергей,3,Registered,0.0,1
2874,Савелий В А,39,Registered,0.0,1
2875,УАВячеслав,17,Registered,1.0,1
2876,Чръный человек,27,Registered,0.0,1


In [89]:
import igraph as ig
import leidenalg


weaponising_ratio_df["article_id"] = weaponising_ratio_df["article"].astype("category").cat.codes
weaponising_ratio_df["user_type"] = weaponising_ratio_df.apply(get_user_type, axis=1)
df_graph = (
    weaponising_ratio_df.groupby(["user", "article_id", "user_type", "weaponising_ratio"])
      .size()
      .reset_index(name="n_edits")
)

pairs = [] # list that will contains set of two users that co-edited the same article

# for each article, we get the set of users that edited it. Then for every user in that set, we create unordered combinaison of 2 users.
for article, group in df_graph.groupby("article_id"):
    users = group["user"].unique()
    for u1, u2 in combinations(sorted(users), 2):
        pairs.append((u1, u2))

coedit_counts = Counter(pairs)

edges_df = pd.DataFrame(
    [(u1, u2, w) for (u1, u2), w in coedit_counts.items()],
    columns=["user1", "user2", "coedit_count"]
)
# Counts how many different articles each pair co-edited (since pairs were added once per article). This count will become the edge weight.

# Optional: remove weak connections (coedit_count < 2)
edges_df = edges_df[edges_df["coedit_count"] >= 3].reset_index(drop=True)

user_stats = (
    df_graph.groupby("user")
    .agg(
        total_edits=("n_edits", "sum"),
        mean_weaponising_ratio=("weaponising_ratio", "mean")
    )
    .reset_index()
)

# Graph from tuple list (handles string names properly)
g = ig.Graph.TupleList(
    edges_df.itertuples(index=False, name=None),  # ensures tuple of (user1, user2, weight)
    weights=True,
    directed=False
)

# Add node attributes
user_dict = user_stats.set_index("user").to_dict("index")

g.vs["name"] = [v["name"] for v in g.vs]
g.vs["total_edits"] = [user_dict.get(v["name"], {}).get("total_edits", 0) for v in g.vs]
g.vs["weaponising_ratio"] = [user_dict.get(v["name"], {}).get("mean_weaponising_ratio", 0) for v in g.vs]

partition = leidenalg.find_partition(
    g, leidenalg.ModularityVertexPartition, weights=g.es["weight"]
)
g.vs["cluster"] = partition.membership

print(f"✅ Graph built: {g.vcount()} nodes, {g.ecount()} edges, {len(set(partition.membership))} clusters.")

✅ Graph built: 75 nodes, 305 edges, 5 clusters.


In [73]:
layout = g.layout_drl()
ig.plot(
    g,
    "../plots/graph_clusters.png",
    layout=layout,
    vertex_size=[5 + 25 * v["weaponising_ratio"] for v in g.vs],
    vertex_color=[v["cluster"] for v in g.vs],
    vertex_label=None,
    edge_width=[min(e["weight"], 5) for e in g.es]
)

AttributeError: Plotting not available; please install pycairo or cairocffi

In [90]:
# Save to GraphML format (recommended for Gephi)
g.write_graphml("../plots/user_network>=3.graphml")
print("✅ Graph exported to '../plots/user_network_w>=5.graphml'")

✅ Graph exported to '../plots/user_network_w>=5.graphml'
